For this code, we set up a new WGAN model with gradient plenaty method. In data pre-training, we also provide new view.

In [ ]:
!pip install iMAP

In [ ]:
!pip install scprep phate magic-impute mnnpy scanpy loompy tbb

In [ ]:
!pip install scikit-misc

In [ ]:
!pip install leidenalg

In [ ]:
pip install git+https://github.com/theislab/scib.git

In [ ]:
import loompy
from numba import jit
from collections import Counter

In [ ]:
import scprep
import imap  #used for feature detected
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import phate
import graphtools as gt
import magic
import os
import datetime
import scanpy as sc
from skmisc.loess import loess
import sklearn.preprocessing as preprocessing

In [ ]:
import umap.umap_ as umap

In [ ]:
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
import torch.utils.data as Data  #Data是用来批训练的模块
from torchvision.utils import save_image
import numpy as np
import os
import pandas as pd
import torch.optim.lr_scheduler as lr_s 

In [ ]:
import rpy2
%load_ext rpy2.ipython

In [ ]:
import scIB
from scipy.spatial.distance import cdist

# Metrics before


In [ ]:
%%R
devtools::install_github('theislab/kBET')

In [ ]:
adata = sc.read_loom('/content/drive/MyDrive/data/cell_lines.loom',sparse=False)

In [ ]:
adata.obs['celltype']

In [ ]:
c = Counter(adata.obs['batch'])
c = dict(c)

In [ ]:
c

In [ ]:
adata.var_names

In [ ]:
label = ['293t' for i in range(len(adata.X))]

In [ ]:
sc.pp.neighbors(adata)

In [ ]:
adata.obs['batch'] = adata.obs['batch'].astype('category')
adata.obs['celltype'] = adata.obs['celltype'].astype('category')
adata.obs['label'] = np.array(label)

In [ ]:
scIB.metrics.graph_connectivity(adata,'celltype') #Graph Conncectivity

In [ ]:
data_umap = umap.UMAP().fit_transform(adata.X)

In [ ]:
label_1 = [adata.obs['celltype'][0] for i in range(len(adata.X))]
label_2 = list(adata.obs['celltype'])
labels = pd.DataFrame()
labels['ref'] = label_1
labels['bat'] = label_2

In [ ]:
%%R -i data_umap -i labels -o batch_estimate

install.packages("devtools")
devtools::install_github("immunogenomics/lisi")

library(lisi)

batch_estimate <- compute_lisi(data_umap, labels, c('ref','bat'))

In [ ]:
print(np.mean(batch_estimate))

In [ ]:
print(np.max(batch_estimate))
print(np.min(batch_estimate))

In [ ]:
label_1 = [adata.obs['batch'][0] for i in range(len(adata.X))]
label_2 = list(adata.obs['batch'])
labels = pd.DataFrame()
labels['ref'] = label_1
labels['bat'] = label_2

In [ ]:
%%R -i data_umap -i labels -o batch_estimate

install.packages("devtools")
devtools::install_github("immunogenomics/lisi")

library(lisi)

batch_estimate <- compute_lisi(data_umap, labels, c('ref','bat'))

In [ ]:
print(np.mean(batch_estimate))

In [ ]:
print(np.max(batch_estimate))
print(np.min(batch_estimate))

In [ ]:
def F1_lisi(blisi,clisi):
  min_val = 1
  max_val = 2
  blisi = (blisi - min_val) / (max_val - min_val)
  clisi = (clisi - min_val) / (max_val - min_val)
  return 2*np.abs(clisi-1)*blisi/(np.abs(clisi-1)+blisi)

In [ ]:
F1_lisi(1.319,1.019)

In [ ]:
scprep.plot.scatter2d(data_umap, c=adata.obs['batch'], figsize=(12,12), cmap="Spectral",
                      ticks=False, label_prefix="UMAP", s = 30)

In [ ]:
scprep.plot.scatter2d(data_umap, c=adata.obs['celltype'], figsize=(12,12), cmap="Spectral",
                      ticks=False, label_prefix="UMAP", s = 30)

In [ ]:
import rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
from sklearn.metrics import silhouette_score
import random
import numpy as np

def silhouette_coeff_ASW(adata, method_use='raw',save_dir='', save_fn='', percent_extract=0.8):
    random.seed(0)
    asw_fscore = []
    asw_bn = []
    asw_bn_sub = []
    asw_ctn = [] 
    iters = []
    for i in range(20):
        iters.append('iteration_'+str(i+1))
        rand_cidx = np.random.choice(adata.obs_names, size=int(len(adata.obs_names) * percent_extract), replace=False)
        print('nb extracted cells: ',len(rand_cidx))
        adata_ext = adata[rand_cidx,:]
        asw_batch = silhouette_score(adata_ext.X, adata_ext.obs['batch'])
        asw_celltype = silhouette_score(adata_ext.X, adata_ext.obs['louvain'])
        min_val = -1
        max_val = 1
        asw_batch_norm = (asw_batch - min_val) / (max_val - min_val)
        asw_celltype_norm = (asw_celltype - min_val) / (max_val - min_val)
        
        fscoreASW = (2 * (1 - asw_batch_norm)*(asw_celltype_norm))/(1 - asw_batch_norm + asw_celltype_norm)
        asw_fscore.append(fscoreASW)
        asw_bn.append(asw_batch_norm)
        asw_bn_sub.append(1-asw_batch_norm)
        asw_ctn.append(asw_celltype_norm)
    
#     iters.append('median_value')
#     asw_fscore.append(np.round(np.median(fscoreASW),3))
#     asw_bn.append(np.round(np.median(asw_batch_norm),3))
#     asw_bn_sub.append(np.round(1 - np.median(asw_batch_norm),3))
#     asw_ctn.append(np.round(np.median(asw_celltype_norm),3))
    df = pd.DataFrame({'asw_batch_norm':asw_bn, 'asw_batch_norm_sub': asw_bn_sub,
                       'asw_celltype_norm': asw_ctn, 'fscore':asw_fscore,
                       'method_use':np.repeat(method_use, len(asw_fscore))})
    df.to_csv(save_dir + save_fn + '.csv')
    print('Save output of pca in: ',save_dir)
    print(df.values.shape)
    print(df.keys())
    return df

In [ ]:
obs = pd.DataFrame()

obs['batch'] = adata.obs['batch']
obs['louvain'] = adata.obs['celltype']

In [ ]:
funcdata = sc.AnnData(data, obs)
silhouette_coeff_ASW(funcdata)

In [ ]:
new_data_1 = data
label = np.array(adata.obs['batch'])

labels = []
for i in adata.obs['batch']:
  labels.append(i)
labels = np.array(labels)
labels = pd.DataFrame(labels)
label = labels

In [ ]:
new_data_1 = adata.X

In [ ]:
%%R -i new_data_1 -i labels -o batch_estimate
library(devtools)
install_github('theislab/kBET')
library(kBET)

print(dim(new_data_1))
print(dim(labels[1]))

batch_estimate <- kBET(new_data_1, labels[[1]])

In [ ]:
for i in batch_estimate:
  print(i)

# Batch Sequencing AWGAN

In [ ]:
adata = imap.stage1.data_preprocess(adata, 'batch')

In [ ]:
data_pre = pd.DataFrame(adata.X)

In [ ]:
adata.X.shape

In [ ]:
count_t = np.array([0,0,0])
for i in adata.obs['batch']:
  if i == 'Jurkat':
    count_t[0]+=1
  if i == '293t':
    count_t[1] += 1
  if i == 'Mix':
    count_t[2] += 1

In [ ]:
labels = ['Jurkat' for i in range(count_t[0])]+['293t' for i in range(count_t[1])]+['Mix' for i in range(count_t[2])]

In [ ]:
#calculate cos distence
@jit(nopython=True)
def pdist(vec1,vec2):
  return vec1@vec2/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
#calculate correlation index
@jit(nopython=True)
def find_correlation_index(frame1, frame2):
  result=[(1,1,1) for i in range(len(frame2))]
  for i in range(len(frame2)):
    distlist=[]
    for j in range(len(frame1)):
      dist = pdist(frame2[i],frame1[j])
      distlist.append((i,j,dist))
    distlist.sort(key=lambda x:x[2],reverse = True)
    result[i] = distlist[0]
  return result

In [ ]:
# A new approach to get the index, what is faster based on our research.
def find_correlation_index(frame1, frame2):
  distlist =  cdist(frame2,frame1,metric='cosine')
  result = np.argmin(distlist,axis=1)
  result2 = []
  for i in range(len(frame2)):
    result2.append((i,result[i]))
  return result2

In [ ]:
def training_set_generator(frame1,frame2,ref,batch):
  common_pair = find_correlation_index(frame1,frame2)
  result = []
  result1 = []
  for i in common_pair:
    result.append(ref[i[1],:])
    result1.append(batch[i[0],:])
  return np.array(result),np.array(result1)

In [ ]:
np.random.seed(999)
torch.manual_seed(999)
torch.cuda.manual_seed_all(999)

In [ ]:
class Mish(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return x*torch.tanh(F.softplus(x))

In [ ]:
#WGAN model, and it does not need to use bath normalization based on WGAN paper.
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(2000, 1024),  
            #nn.BatchNorm1d(1024),
            Mish(),
            nn.Linear(1024, 512),  
            #nn.BatchNorm1d(512),
            Mish(),
            nn.Linear(512, 256),  
            #nn.BatchNorm1d(256),
            Mish(),
            nn.Linear(256, 128),  
            #nn.BatchNorm1d(128),
            Mish(),
            nn.Linear(128, 1)

        )

    def forward(self, x):
        x = self.dis(x)
        return x
 
 
# WGAN generator
# Require batch normalization
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.relu_f = nn.ReLU(True)
        self.gen = nn.Sequential(
            nn.Linear(2000, 1024),
            nn.BatchNorm1d(1024),
            Mish(),

            nn.Linear(1024, 512),  
            nn.BatchNorm1d(512),
            Mish(),

            nn.Linear(512, 256),  
            nn.BatchNorm1d(256),
            Mish(),

            nn.Linear(256, 512),  
            nn.BatchNorm1d(512),
            Mish(),
  

            nn.Linear(512, 1024),  
            nn.BatchNorm1d(1024),
            Mish(),

            nn.Linear(1024, 2000),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        gre = self.gen(x)
        return self.relu_f(gre+x)    #residual network
 

In [ ]:
# calculate gradient penalty
def calculate_gradient_penalty(real_data, fake_data, D): 
  eta = torch.FloatTensor(real_data.size(0),1).uniform_(0,1) 
  eta = eta.expand(real_data.size(0), real_data.size(1)) 
  cuda = True if torch.cuda.is_available() else False 
  if cuda: 
    eta = eta.cuda() 
  else: 
    eta = eta 
  interpolated = eta * real_data + ((1 - eta) * fake_data) 
  if cuda: 
    interpolated = interpolated.cuda() 
  else: 
    interpolated = interpolated 
   # define it to calculate gradient 
  interpolated = Variable(interpolated, requires_grad=True) 
   # calculate probability of interpolated examples 
  prob_interpolated = D(interpolated) 
  # calculate gradients of probabilities with respect to examples 
  gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated, 
  grad_outputs=torch.ones( 
  prob_interpolated.size()).cuda() if cuda else torch.ones( 
  prob_interpolated.size()), 
  create_graph=True, retain_graph=True)[0] 
  grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() 
  return grad_penalty

In [ ]:
@jit(nopython = True)
def determine_batch(val1):
  val_list =[32,64,128,256]
  for i in val_list:
    if val1%i !=1:
      return i
    else:
      continue
  return val1

In [ ]:
def WGAN_train(train_label,train_data,epoch,batch,lambda_1):
  stop = 0
  iter = 0
  D = discriminator()
  G = generator()

  if torch.cuda.is_available():
    D = D.cuda()
    G = G.cuda()
  lr=0.0001

  d_optimizer = torch.optim.AdamW(D.parameters(), lr=lr)
  g_optimizer = torch.optim.AdamW(G.parameters(), lr=lr)  

  G.train()
  D.train()

  MAX_ITER = train_data.shape[0]
  batch = determine_batch(MAX_ITER)

  for epoch_1 in range(epoch):
    print("This is ", epoch_1)
    for time in range(0,MAX_ITER,batch):
      true_data = torch.FloatTensor(train_label[time:time+batch,:]).cuda()
      false_data = torch.FloatTensor(train_data[time:time+batch,:]).cuda()
    

      #train d at first

      d_optimizer.zero_grad()

      real_out = D(true_data)
      real_label_loss = -torch.mean(real_out)


      # train use WGAN

      fake_out_new = G(false_data).detach()
      fake_out = D(fake_out_new)

      div = calculate_gradient_penalty(true_data, false_data, D)

      label_loss = real_label_loss+torch.mean(fake_out)+div/lambda_1
      label_loss.backward()


      d_optimizer.step()

  
      #train G

      real_out = G(false_data)
      real_output = D(real_out)

      real_loss1 = -torch.mean(real_output)

      g_optimizer.zero_grad()

      real_loss1.backward()
      g_optimizer.step()
      # scheduler_G.step()

      if(time%100==0):
        print("g step loss",real_loss1)
      iter += 1
  print("Train step finished")
  G.eval()
  test_data = torch.FloatTensor(train_data).cuda()
  test_list = G(test_data).detach().cpu().numpy()    
  return test_list,G

In [ ]:
def sequencing_train(ref_adata, batch_adata, batch_inf, epoch=300, batch=32, lambda_1=1):
  ref_data_ori = ref_adata.X
  for bat_inf in batch_inf[1:]:
    print("##########################Training%s#####################"%(bat_inf))
    batch_data_ori = batch_adata[batch_adata.obs['batch'] == bat_inf].X
    label_data,train_data = training_set_generator(ref_data_ori, batch_data_ori, ref_data_ori, batch_data_ori)
    print("#################Finish Pair finding##########################")
    remove_batch_data,G_tar = WGAN_train(label_data,train_data,epoch,batch,lambda_1)
    ref_data_ori = np.vstack([ref_data_ori,remove_batch_data])
  print("###################### Finish Training ###########################")
  return ref_data_ori, G_tar

In [ ]:
adata.obs['batch']

In [ ]:
ref_adata = adata[adata.obs['batch'] == adata.obs['batch'][0]]

batch_adata = adata[adata.obs['batch'] != adata.obs['batch'][0]]


In [ ]:
c = dict(Counter(adata.obs['batch']))
c_l=list(c.keys())

In [ ]:
data,G_new = sequencing_train(ref_adata,batch_adata,batch_inf=c_l)

In [ ]:
data

In [ ]:
import umap.umap_ as umap

In [ ]:
data_umap = umap.UMAP().fit_transform(data)

In [ ]:
scprep.plot.scatter2d(data_umap, c=adata.obs['batch'], figsize=(12,12), cmap="Spectral",
                      ticks=False, label_prefix="UMAP", s = 30)

In [ ]:
scprep.plot.scatter2d(data_umap, c=adata.obs['celltype'], figsize=(12,12), cmap="Spectral",
                      ticks=False, label_prefix="UMAP", s = 30)

In [ ]:
adata_new = sc.AnnData(data)
adata_new.obs['batch'] = np.array(list(adata.obs['batch']))
adata_new.obs['celltype'] = np.array(list(adata.obs['celltype']))
adata_new.var_names = adata.var_names

# Metrics after

In [ ]:
from sklearn.metrics import silhouette_score
import random
import numpy as np

def silhouette_coeff_ASW(adata, method_use='raw',save_dir='', save_fn='', percent_extract=0.8):
    random.seed(0)
    asw_fscore = []
    asw_bn = []
    asw_bn_sub = []
    asw_ctn = [] 
    iters = []
    for i in range(20):
        iters.append('iteration_'+str(i+1))
        rand_cidx = np.random.choice(adata.obs_names, size=int(len(adata.obs_names) * percent_extract), replace=False)
        print('nb extracted cells: ',len(rand_cidx))
        adata_ext = adata[rand_cidx,:]
        asw_batch = silhouette_score(adata_ext.X, adata_ext.obs['batch'])
        asw_celltype = silhouette_score(adata_ext.X, adata_ext.obs['louvain'])
        min_val = -1
        max_val = 1
        asw_batch_norm = (asw_batch - min_val) / (max_val - min_val)
        asw_celltype_norm = (asw_celltype - min_val) / (max_val - min_val)
        
        fscoreASW = (2 * (1 - asw_batch_norm)*(asw_celltype_norm))/(1 - asw_batch_norm + asw_celltype_norm)
        asw_fscore.append(fscoreASW)
        asw_bn.append(asw_batch_norm)
        asw_bn_sub.append(1-asw_batch_norm)
        asw_ctn.append(asw_celltype_norm)
    
#     iters.append('median_value')
#     asw_fscore.append(np.round(np.median(fscoreASW),3))
#     asw_bn.append(np.round(np.median(asw_batch_norm),3))
#     asw_bn_sub.append(np.round(1 - np.median(asw_batch_norm),3))
#     asw_ctn.append(np.round(np.median(asw_celltype_norm),3))
    df = pd.DataFrame({'asw_batch_norm':asw_bn, 'asw_batch_norm_sub': asw_bn_sub,
                       'asw_celltype_norm': asw_ctn, 'fscore':asw_fscore,
                       'method_use':np.repeat(method_use, len(asw_fscore))})
    df.to_csv(save_dir + save_fn + '.csv')
    print('Save output of pca in: ',save_dir)
    print(df.values.shape)
    print(df.keys())
    return df

In [ ]:
obs = pd.DataFrame()

obs['batch'] = adata.obs['batch']
obs['louvain'] = adata.obs['celltype']

In [ ]:
funcdata = sc.AnnData(data, obs)
silhouette_coeff_ASW(funcdata)

In [ ]:
new_data_1 = data
label = np.array(adata.obs['batch'])

labels = []
for i in adata.obs['batch']:
  labels.append(i)
labels = np.array(labels)
labels = pd.DataFrame(labels)
label = labels

In [ ]:
import rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i new_data_1 -i labels -o batch_estimate
library(devtools)
install_github('theislab/kBET')
library(kBET)

print(dim(new_data_1))
print(dim(labels[1]))

batch_estimate <- kBET(new_data_1, labels[[1]])

In [ ]:
for i in batch_estimate:
  print(i)

In [ ]:
data_umap = umap.UMAP().fit_transform(data)

In [ ]:
label_1 = ['Jurkat' for i in range(len(adata.X))]
label_2 = list(adata.obs['celltype'])
labels = pd.DataFrame()
labels['ref'] = label_1
labels['bat'] = label_2

In [ ]:
%%R -i data_umap -i labels -o batch_estimate

install.packages("devtools")
devtools::install_github("immunogenomics/lisi")

library(lisi)

batch_estimate <- compute_lisi(data_umap, labels, c('ref','bat'))

In [ ]:
print(np.mean(batch_estimate))

In [ ]:
print(np.max(batch_estimate))
print(np.min(batch_estimate))

In [ ]:
label_1 = ['Jurkat' for i in range(len(adata.X))]
label_2 = list(adata.obs['batch'])
labels = pd.DataFrame()
labels['ref'] = label_1
labels['bat'] = label_2

In [ ]:
%%R -i data_umap -i labels -o batch_estimate

install.packages("devtools")
devtools::install_github("immunogenomics/lisi")

library(lisi)

batch_estimate <- compute_lisi(data_umap, labels, c('ref','bat'))

In [ ]:
print(np.mean(batch_estimate))

In [ ]:
print(np.max(batch_estimate))
print(np.min(batch_estimate))

In [ ]:
scIB.metrics.graph_connectivity(funcdata, 'louvain')